In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer
import json
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.decomposition import PCA

from sklearn.metrics.pairwise import cosine_similarity
import ast
import datetime
import tensorflow as tf



In [23]:
import re
%pip install scikeras
from scikeras.wrappers import KerasClassifier
from typing import Tuple, List, Union, Optional
from networkx import scale_free_graph
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from transformers import BertTokenizer
import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
import datetime
from scipy.spatial.distance import cosine
from imblearn.over_sampling import SMOTE

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
class ClassicalModels:
    """A class for processing and modeling code similarity data using classical ML approaches."""
    
    def __init__(self, data: pd.DataFrame, target: str, max_length: int = 500):
        """
        Initialize the ClassicalModels class.
        
        Args:
            data: Input DataFrame containing code pairs
            target: Target variable column name
            max_length: Maximum sequence length for token padding
        """
        self.data = data
        self.target = target
        self.max_length = max_length
        self.model: Optional[Pipeline] = None
        self.batch_size = 32
        self.epochs=50
        
        self.X_cosine=None
        self.y=None
        
        
        # Configure logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    @staticmethod
    def remove_java_string(text: str) -> str:
        """
        Remove Java code block markers from string.
        
        Args:
            text: Input string containing Java code markers
            
        Returns:
            String with Java markers removed
        """
        return text.replace("```java", "").replace("```", "")

    def pad_sequence(self, tokens: List[int], max_length: Optional[int] = None) -> np.ndarray:
        """
        Pad or truncate token sequence to fixed length.
        
        Args:
            tokens: List of token IDs
            max_length: Desired sequence length (defaults to self.max_length)
            
        Returns:
            Padded numpy array of tokens
        """
        max_length = max_length or self.max_length
        tokens = np.array(tokens)
        
        if len(tokens) < max_length:
            return np.pad(tokens, (0, max_length - len(tokens)), mode='constant')
        return tokens[:max_length]

    def preprocess_data(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, List, List]:

        try:
            # Clean data
            if 'Unnamed: 0' in df.columns:
                df.drop(['Unnamed: 0'], axis=1, inplace=True)
            
            print(f"Processing DataFrame with shape: {df.shape}")
            
            # Clean Java strings
            df['java_translation'] = df['java_translation'].apply(
                lambda x: self.remove_java_string(str(x)) if isinstance(x, list) else self.remove_java_string(x)
            )
            
            # Process token sequences
            df['java_features'] = df['java_tokens'].apply(
                lambda x: self.pad_sequence(eval(x) if isinstance(x, str) else x)
            )
            df['python_features'] = df['python_tokens'].apply(
                lambda x: self.pad_sequence(eval(x) if isinstance(x, str) else x)
            )
            
            # Prepare features and target
            features = np.array(df['java_features'].tolist() + df['python_features'].tolist())
            # target = df['Is_Equal'].tolist() * 2
            target = np.array(df['Is_Equal'].tolist() * 2)
            
            #Scale Features
            scaler=StandardScaler()
            features=scaler.fit_transform(features)
            

            return features, target
            
        except Exception as e:
            self.logger.error(f"Error in preprocessing data: {str(e)}")
            raise
    def engineer_features(self, df: pd.DataFrame) -> np.ndarray:
        # Parse embeddings if needed
        df['python_tokens'] = df['python_tokens'].apply(
            lambda x: np.array(ast.literal_eval(x) if isinstance(x, str) else x)
        )
        df['java_tokens'] = df['java_tokens'].apply(
            lambda x: np.array(ast.literal_eval(x) if isinstance(x, str) else x)
        )

        # Pad sequences
        df['python_tokens'] = df['python_tokens'].apply(self.pad_sequence)
        df['java_tokens'] = df['java_tokens'].apply(self.pad_sequence)

        # Feature Engineering
        # 1. Absolute difference
        abs_diff = df.apply(
            lambda row: np.abs(row['python_tokens'] - row['java_tokens']), axis=1
        )
        X_abs_diff = np.stack(abs_diff)

        # 2. Cosine similarity
        cos_sim = df.apply(
            lambda row: cosine_similarity([row['python_tokens']], [row['java_tokens']])[0][0],
            axis=1
        )
        X_cos_sim = np.array(cos_sim).reshape(-1, 1)

        # 3. Concatenated embeddings
        concat_tokens = df.apply(
            lambda row: np.concatenate((row['python_tokens'], row['java_tokens'])),
            axis=1
        )
        X_concat = np.stack(concat_tokens)

        # Combine features
        X = np.hstack((X_abs_diff, X_cos_sim, X_concat))

        return X
    def preprocess_with_pca(self, df: pd.DataFrame, n_components: int = 60) -> Tuple[np.ndarray, np.ndarray]:

        try:
            # Ensure tokens are properly formatted
            df['java_tokens'] = df['java_tokens'].apply(
                lambda x: eval(x) if isinstance(x, str) else x
            )
            df['python_tokens'] = df['python_tokens'].apply(
                lambda x: eval(x) if isinstance(x, str) else x
            )
            
            # Pad sequences
            df['java_tokens'] = df['java_tokens'].apply(
                lambda x: self.pad_sequence(x)
            )
            df['python_tokens'] = df['python_tokens'].apply(
                lambda x: self.pad_sequence(x)
            )
            
            # Convert to numpy arrays
            X_java = np.array(df['java_tokens'].tolist())
            X_python = np.array(df['python_tokens'].tolist())
            
            # Scale features
            scaler = StandardScaler()
            X_java = scaler.fit_transform(X_java)
            X_python = scaler.fit_transform(X_python)
            
            # Apply PCA
            pca = PCA(n_components=n_components, random_state=42)
            X_java_pca = pca.fit_transform(X_java)
            X_python_pca = pca.fit_transform(X_python)
            
            # Calculate cosine similarities
            cosine_similarities = []
            for i in range(len(X_java_pca)):
                similarity = 1 - cosine(X_java_pca[i], X_python_pca[i])
                cosine_similarities.append(similarity)
            
            self.X_cosine = np.array(cosine_similarities).reshape(-1, 1)
            self.y = df[self.target].values
            
            return self.X_cosine, self.y
            
        except Exception as e:
            self.logger.error(f"Error in PCA preprocessing: {str(e)}")
            raise


    def tokenize_data(self, df: pd.DataFrame) -> None:
        """
        Tokenize Java and Python code using BERT tokenizers.
        
        Args:
            df: Input DataFrame containing code pairs
        """
        try:
            # Tokenize Java code
            java_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            df['java_translation'] = df['java_translation'].apply(
                lambda x: java_tokenizer.encode(x, add_special_tokens=True)
            )
            
            # Tokenize Python code
            python_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
            df['java_tokens'] = df['java_translation'].apply(
                lambda x: python_tokenizer.encode(x, add_special_tokens=True)
            )
            df['python_tokens'] = df['original_code'].apply(
                lambda x: python_tokenizer.encode(x, add_special_tokens=True)
            )
            
            # Convert target to int
            df['Is_Equal'] = df['Is_Equal'].astype(int)
            
        except Exception as e:
            self.logger.error(f"Error in tokenizing data: {str(e)}")
            raise


    def train_random_forest(self,features,target):
        try:
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                features, target, test_size=0.2, random_state=42
            )
            
            # Create and train pipeline
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('clf', RandomForestClassifier(
                    criterion='entropy',
                    max_depth=8,
                    max_features='sqrt',
                    min_samples_leaf=8,
                    min_samples_split=10
                ))
            ])
            
            pipeline.fit(X_train, y_train)
            self.model = pipeline
            
            # Generate predictions
            y_pred = pipeline.predict(X_test)
            
            # Log performance metrics
            print(f"RF  Accuracy: {accuracy_score(y_test, y_pred):.4f}")
            print(f"RF Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
            print(f"Rf Classification Report:\n{classification_report(y_test, y_pred)}")
            print(f"RF Train Score: {pipeline.score(X_train, y_train):.4f}")
            print(f"RF Test Score: {pipeline.score(X_test, y_test):.4f}")
            
            return pipeline
            
        except Exception as e:
            self.logger.error(f"Error in training random forest: {str(e)}")
            raise
        
    def neural_network(self,features,target) :
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
        # Define the model
        model = Sequential([
            Dense(128, input_shape=(self.max_length,), activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(1, activation='sigmoid')
        ])
                    # Compile the model
        model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=0.01),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )

            # Define callbacks
        lr_scheduler = ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.7,
                patience=2,
                min_lr=0.00001
            )

            # TensorBoard logging
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = TensorBoard(
                log_dir=log_dir,
                histogram_freq=1
            )

            # Train the model
        print("Starting neural network training...")
        history = model.fit(
                X_train,
                y_train,
                epochs=self.epochs,
                batch_size=self.batch_size,
                validation_data=(X_test, y_test),
                callbacks=[lr_scheduler, tensorboard_callback]
            )

            # Evaluate the model
        train_loss, train_accuracy = model.evaluate(X_train, y_train)
        test_loss, test_accuracy = model.evaluate(X_test, y_test)

        print(f"NN Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"NN Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

        # Generate predictions for test set
        y_pred = model.predict(X_test) > 0.5
        print(f"NN Classification Report:\n{classification_report(y_test, y_pred)}")
        print(f"NN Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")

        return model    

    def GBoost(self,features, target):
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
        model= GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, max_depth=10, random_state=42)
        model.fit(X_train, y_train)
        y_pred=model.predict(X_test)
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
        print(f"Classification Report:\n{classification_report(y_test, y_pred)}")
        return model

    def create_ensemble(self, features, target) -> VotingClassifier:
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
        print("Creating model ensemble...")

        # Train individual models
        rf_model = self.train_random_forest(X_train, X_test, y_train, y_test)
        nn_model = KerasClassifier(build_fn=lambda: self.neural_network(X_train, X_test, y_train, y_test), epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        gboost_model = self.GBoost(X_train, X_test, y_train, y_test)

        # Create a VotingClassifier
        ensemble_model = VotingClassifier(
            estimators=[
                ('rf', rf_model.named_steps['clf']),
                ('gboost', gboost_model),
                ('nn', nn_model)
            ],
            voting='soft'
        )

        # Fit the ensemble model
        ensemble_model.fit(X_train, y_train)
        self.model = ensemble_model

        # Generate predictions
        y_pred = ensemble_model.predict(X_test)

        # Log performance metrics
        
        print(f"Ensemble Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"Ensemble Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
        print(f"Ensemble Classification Report:\n{classification_report(y_test, y_pred)}")

        return ensemble_model

    
    def train_knn_with_smote(self, X: np.ndarray, y: np.ndarray) -> KNeighborsClassifier:

        try:
            # Apply SMOTE
            smote = SMOTE(random_state=self.random_seed)
            X_resampled, y_resampled = smote.fit_resample(X, y)
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X_resampled, y_resampled, test_size=0.2, random_state=self.random_seed
            )
            
            # Define hyperparameter grid
            param_grid = {
                'n_neighbors': [3, 5, 7, 9, 11],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan']
            }
            
            # Perform GridSearchCV
            grid = GridSearchCV(
                KNeighborsClassifier(),
                param_grid,
                cv=5,
                scoring='accuracy',
                n_jobs=-1
            )
            grid.fit(X_train, y_train)
            
            # Get best model
            knn = grid.best_estimator_
            
            # Print results
            print(f"Best Hyperparameters: {grid.best_params_}")
            
            # Calculate and print metrics
            y_train_pred = knn.predict(X_train)
            y_test_pred = knn.predict(X_test)
            
            print(f"KNN Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
            print(f"KNN Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
            print("KNN Classification Report (Test):")
            print(classification_report(y_test, y_test_pred))
            
            # Store the model
            self.X_train, self.X_test = X_train, X_test
            self.y_train, self.y_test = y_train, y_test
            
            return knn
            
        except Exception as e:
            self.logger.error(f"Error in KNN training: {str(e)}")
            raise
    
    def train_svm(self, X: np.ndarray, y: np.ndarray, n_components: int = 400) -> SVC:

        try:
            # Reduce dimensionality with PCA
            pca = PCA(n_components=n_components, random_state=self.random_seed)
            X = pca.fit_transform(X)
            
            # Apply SMOTE
            smote = SMOTE(random_state=self.random_seed)
            X, y = smote.fit_resample(X, y)
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=self.random_seed, stratify=y
            )
            
            # Store splits
            self.X_train, self.X_test = X_train, X_test
            self.y_train, self.y_test = y_train, y_test
            
            # Define parameter grid
            param_grid = {
                'C': [0.1, 1, 10, 100],
                'kernel': ['linear', 'rbf', 'poly'],
                'gamma': [0.01, 0.1],
                'degree': [2, 3, 4]
            }
            
            # Initialize SVM
            svm = SVC(class_weight='balanced', random_state=self.random_seed)
            
            # Perform GridSearchCV
            grid_search = GridSearchCV(
                svm,
                param_grid,
                cv=5,
                scoring='accuracy',
                verbose=0,
                n_jobs=-1
            )
            grid_search.fit(X_train, y_train)
            
            # Get best model
            best_svm = grid_search.best_estimator_
            
            # Cross-validation
            cv_scores = cross_val_score(best_svm, X_train, y_train, cv=5)
            
            # Predictions
            train_pred = best_svm.predict(X_train)
            test_pred = best_svm.predict(X_test)
            
            # Print results
            print(f"Best Parameters: {grid_search.best_params_}")
            print(f"Cross-validation scores: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
            print(f"SVM Train Accuracy: {accuracy_score(y_train, train_pred) * 100:.2f}%")
            print(f"SVM Test Accuracy: {accuracy_score(y_test, test_pred) * 100:.2f}%")
            print("\nClassification Report:")
            print(classification_report(y_test, test_pred))
            
            return best_svm
            
        except Exception as e:
            self.logger.error(f"Error in SVM training: {str(e)}")
            raise
    

data=pd.read_csv('combined_df.csv')
models=ClassicalModels(data, 'Is_Equal')
models.tokenize_data(data)


# features, target = models.preprocess_data(data)
# models.train_random_forest(features,target)
# models.neural_network(features,target)
# models.GBoost(features,target)


models.preprocess_with_pca(data)
models.train_knn_with_smote(models.X_cosine, models.y)

models.feature_engineering(data)
models.train_svm(models.X, models.y)





ImportError: cannot import name '_get_column_indices' from 'sklearn.utils' (c:\Users\marya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\__init__.py)

In [19]:
# import pickle


# # pickle.dump(DTpipeline, open('DTpipeline.pkl', 'wb'))
# pickle.dump(NNpipeline, open('NNpipeline.pkl', 'wb'))

